## Waterbody Data Extraction from NLCD Data

<b>NLCD Data Download Link:</b> <a href = "https://www.mrlc.gov/viewer/">  Multi-Resolution Land Characteristics (MRLC) Viewer</a>

<b> Census Block Data Download Link:</b> <a href="https://www.census.gov/cgi-bin/geo/shapefiles/index.php"> Tiger/Line Shapefiles</a>

### Required Packages Loading

In [1]:
import geopandas as gpd
import pandas as pd
import rasterio
from pygris import blocks
from rasterstats import zonal_stats
import glob
import os
import tifffile
import mapclassify
import re

### Reading Data

In [5]:
# Shapeflie Path
charlotte_blocks = gpd.read_file("census_block_clt.shp") #Change this file path according to your census block shapefile

folder_path = r"C:\Users\jbiswas\UNC Charlotte Dropbox\Jayanta Biswas\Wetland_CLT\CLT\NLCD" #Change where you stored your raster files

In [6]:
years = ["01", "02","03", "04", "05","06",
        "07", "08","09","10", "11","12",
        "13", "14","15","16", "17","18",
        "19", "20","21","22", "23","24",]

### Checking Data

In [7]:
# Identify all NLCD files in your folder
raster_files = [f for f in os.listdir(folder_path) if f.startswith('Annual_NLCD_LndCov')] #Change according to your data

In [13]:
raster_files

['Annual_NLCD_LndCov_2001_CU_C1V1_5f54d194-97be-4ff1-85ca-f389d71f37b3.tiff',
 'Annual_NLCD_LndCov_2002_CU_C1V1_5f54d194-97be-4ff1-85ca-f389d71f37b3.tiff',
 'Annual_NLCD_LndCov_2003_CU_C1V1_5f54d194-97be-4ff1-85ca-f389d71f37b3.tiff',
 'Annual_NLCD_LndCov_2004_CU_C1V1_5f54d194-97be-4ff1-85ca-f389d71f37b3.tiff',
 'Annual_NLCD_LndCov_2005_CU_C1V1_5f54d194-97be-4ff1-85ca-f389d71f37b3.tiff',
 'Annual_NLCD_LndCov_2006_CU_C1V1_5f54d194-97be-4ff1-85ca-f389d71f37b3.tiff',
 'Annual_NLCD_LndCov_2007_CU_C1V1_5f54d194-97be-4ff1-85ca-f389d71f37b3.tiff',
 'Annual_NLCD_LndCov_2008_CU_C1V1_5f54d194-97be-4ff1-85ca-f389d71f37b3.tiff',
 'Annual_NLCD_LndCov_2009_CU_C1V1_5f54d194-97be-4ff1-85ca-f389d71f37b3.tiff',
 'Annual_NLCD_LndCov_2010_CU_C1V1_5f54d194-97be-4ff1-85ca-f389d71f37b3.tiff',
 'Annual_NLCD_LndCov_2011_CU_C1V1_5f54d194-97be-4ff1-85ca-f389d71f37b3.tiff',
 'Annual_NLCD_LndCov_2012_CU_C1V1_5f54d194-97be-4ff1-85ca-f389d71f37b3.tiff',
 'Annual_NLCD_LndCov_2013_CU_C1V1_5f54d194-97be-4ff1-85ca-f389d7

In [14]:
print("CRS:", charlotte_blocks.crs)

CRS: EPSG:4269


In [15]:
with rasterio.open(os.path.join(folder_path, raster_files[0])) as src:
    print("CRS:", src.crs)

CRS: PROJCS["AEA        WGS84",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",23],PARAMETER["longitude_of_center",-96],PARAMETER["standard_parallel_1",29.5],PARAMETER["standard_parallel_2",45.5],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]


### Align Coordinate Reference Systems (CRS)

In [16]:
# Align CRS (Using the first file as reference)
with rasterio.open(os.path.join(folder_path, raster_files[0])) as src:
    charlotte_blocks = charlotte_blocks.to_crs(src.crs)

### Run Categorical Zonal Statistics and Calculating Area

In [17]:
all_results = []

# 3. Process each year
for file_name in sorted(raster_files):
    # Extract year using Regex: look for 4 digits after 'LndCov_'
    match = re.search(r'LndCov_(\d{4})', file_name)
    if not match:
        continue
    year = match.group(1)
    
    print(f"Processing Year: {year}...")
    file_path = os.path.join(folder_path, file_name)
    
    # Run categorical stats
    stats = zonal_stats(charlotte_blocks, file_path, categorical=True)
    
    # 4. Parse statistics for each census block
    for i, entry in enumerate(stats):
        # Pixel counts (NLCD classes: 11=Water, 90=Woody Wetland, 95=Emergent Wetland)
        c11 = entry.get(11, 0)
        c90 = entry.get(90, 0)
        c95 = entry.get(95, 0)
        
        all_results.append({
            'GEOID20': charlotte_blocks.iloc[i]['GEOID'],
            'year': int(year),
            'water_m2': c11 * 900,
            'wetland_woody_m2': c90 * 900,
            'wetland_herb_m2': c95 * 900,
            'total_wet_area_m2': (c11 + c90 + c95) * 900
        })
# 5. Create Master DataFrame
df_long = pd.DataFrame(all_results)

Processing Year: 2001...
Processing Year: 2002...
Processing Year: 2003...
Processing Year: 2004...
Processing Year: 2005...
Processing Year: 2006...
Processing Year: 2007...
Processing Year: 2008...
Processing Year: 2009...
Processing Year: 2010...
Processing Year: 2011...
Processing Year: 2012...
Processing Year: 2013...
Processing Year: 2014...
Processing Year: 2015...
Processing Year: 2016...
Processing Year: 2017...
Processing Year: 2018...
Processing Year: 2019...
Processing Year: 2020...
Processing Year: 2021...
Processing Year: 2022...
Processing Year: 2023...
Processing Year: 2024...


### Check the Output

In [19]:
df_long.head()

,GEOID20,year,water_m2,wetland_woody_m2,wetland_herb_m2,total_wet_area_m2
0,37119005306,2001,900,0,0,900
1,37119005307,2001,0,0,0,0
2,37119001508,2001,77400,10800,0,88200
3,37119005613,2001,57600,12600,0,70200
4,37119006214,2001,4500,332100,0,336600


In [20]:
# 6. Create a Summary (Total water area in Mecklenburg County over time)
summary = df_long.groupby('year')[['water_m2', 'total_wet_area_m2']].sum() / 1_000_000
summary.columns = ['Water (sq km)', 'Total Wet Area (sq km)']

print("\n--- Summary for Mecklenburg County ---")
print(summary)


--- Summary for Mecklenburg County ---
      Water (sq km)  Total Wet Area (sq km)
year                                       
2001        52.5789                 65.5479
2002        52.1613                 65.1204
2003        53.1801                 66.0501
2004        52.8822                 65.7378
2005        52.6869                 65.5317
2006        52.2387                 65.0844
2007        51.3108                 64.1709
2008        51.9480                 64.7316
2009        51.8373                 64.6092
2010        51.9156                 64.6587
2011        51.9813                 64.7199
2012        51.9282                 64.6560
2013        52.1190                 64.8414
2014        52.1892                 64.8963
2015        51.8967                 64.5642
2016        52.2252                 64.8603
2017        52.4241                 64.9935
2018        52.0722                 64.6389
2019        51.9759                 64.5282
2020        51.8040                 

### Saving The Data

In [21]:
# Save results
df_long.to_csv("clt_water_time_series_2001_2024.csv", index=False)

### Prepare the Data for Mapping

#### Wetland (Wetland Woody + Wetland Herb)

In [23]:
df_long['Wetland_m2'] = df_long['wetland_woody_m2'] + df_long['wetland_herb_m2']
df_long.head()

,GEOID20,year,water_m2,wetland_woody_m2,wetland_herb_m2,total_wet_area_m2,Wetland_m2
0,37119005306,2001,900,0,0,900,0
1,37119005307,2001,0,0,0,0,0
2,37119001508,2001,77400,10800,0,88200,10800
3,37119005613,2001,57600,12600,0,70200,12600
4,37119006214,2001,4500,332100,0,336600,332100


In [25]:
# Pivot the data to wide format for Wetland (woody + herb)
wetland_wide = df_long.pivot(index='GEOID20', columns='year', values='Wetland_m2').reset_index()
wetland_wide.to_csv('wetland_wide.csv', index=False)

In [26]:
wetland_wide.head()

year,GEOID20,2001,2002,2003,2004,2005,2006,2007,2008,2009,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,37119000101,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,37119000102,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,37119000103,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,37119000104,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,37119000301,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Open Water

In [27]:
# Pivot the data to wide format for Water
water_wide = df_long.pivot(index='GEOID20', columns='year', values='water_m2').reset_index()
water_wide.to_csv('water_wide.csv', index=False)

In [28]:
water_wide.head()

year,GEOID20,2001,2002,2003,2004,2005,2006,2007,2008,2009,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,37119000101,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,37119000102,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,37119000103,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,37119000104,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,37119000301,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Total Wetland

In [29]:
# Pivot the data to wide format for Total Wet Area
total_wet_wide = df_long.pivot(index='GEOID20', columns='year', values='total_wet_area_m2').reset_index()
total_wet_wide.to_csv('total_wet_area_wide.csv', index=False)

In [30]:
total_wet_wide.head()

year,GEOID20,2001,2002,2003,2004,2005,2006,2007,2008,2009,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,37119000101,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,37119000102,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,37119000103,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,37119000104,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,37119000301,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
